<a href="https://colab.research.google.com/github/jayasuneja/FWE458_Spring25/blob/main/Homework4_Suneja/Homework4_Suneja.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework 4: Linear Regression Model with SciPy Optimization and Statistics Objective

In this assignment, you will apply linear regression techniques using SciPy's optimization and statistical functions to model Gross Primary Productivity (GPP) based on environmental factors. You will analyze how Shortwave Radiation (SW), Vapor Pressure Deficit (VPD), and Air Temperature (TA) impact GPP. Additionally, you will include interaction terms to account for potential variable interactions.

Dataset: **Wcr_GPPdaily.csv**
* Contains daily measurements of GPP and environmental factors
  * TIMESTAMP: Date of observation (YYYYMMDD)
  * TA_F: Air Temperature [C]
  * SW_IN_F: Shortwave Radiation [W/m2]
  * VPD_F: Vapor Pressure Deficit [kPa]
  * GPP_NT_VUT_REF: Gross Primary Productivity [gC/m2/day]

* For this assignment, rename the columns as follows:
  * TA_F -- TA
  * SW_IN_F -- SW
  * VPD_F -- VPD
  * GPP_NT_VUT_REF -- GPP (Target Variable)

# Tasks


# 1) Data Preprocessing (1 point)

## 1a) Load the data using pandas and display the first few rows

In [ ]:
# mounting drive
from google.colab import drive
drive.mount('/content/drive')
filedir = '/content/drive/MyDrive/FWE458/'
drive.mount("/content/drive", force_remount=True)

# importing dataset
import pandas as pd
fname = filedir + "Wcr_GPPdaily.csv"
DF = pd.read_csv(fname)

# printing DF head
print(DF.head())

Mounted at /content/drive
Mounted at /content/drive
   TIMESTAMP    TA_F  SW_IN_F  VPD_F  GPP_NT_VUT_REF
0   19990101 -20.063   72.603  0.413       -0.517364
1   19990102 -12.814   12.358  0.147       -0.094241
2   19990103 -12.625   33.132  0.128       -0.166819
3   19990104 -18.652   93.481  0.263       -0.582301
4   19990105 -20.269   45.502  0.261       -0.568240


## 1b) Rename columns

In [ ]:
# Renaming TA_F -- TA
DF.rename(columns = {"TA_F":"TA"}, inplace = True)

# Renaming SW_IN_F -- SW
DF.rename(columns = {"SW_IN_F":"SW"}, inplace = True)

# Renaming VPD_F -- VPD
DF.rename(columns = {"VPD_F":"VPD"}, inplace = True)

# Renaming GPP_NT_VUT_REF -- GPP (target variable)
DF.rename(columns = {"GPP_NT_VUT_REF":"GPP"}, inplace = True)

# Displaying new column titles
print(DF.head())  # sucess!

   TIMESTAMP      TA      SW    VPD       GPP
0   19990101 -20.063  72.603  0.413 -0.517364
1   19990102 -12.814  12.358  0.147 -0.094241
2   19990103 -12.625  33.132  0.128 -0.166819
3   19990104 -18.652  93.481  0.263 -0.582301
4   19990105 -20.269  45.502  0.261 -0.568240


## 1c) Drop the TIMESTAMP column as it is not needed for regression

In [ ]:
# dropping TIMESTAMP column
DF.drop(columns = ["TIMESTAMP"], inplace = True, errors = "ignore")

# displaying new dataframe
print(DF.head())  # sucess!

       TA      SW    VPD       GPP
0 -20.063  72.603  0.413 -0.517364
1 -12.814  12.358  0.147 -0.094241
2 -12.625  33.132  0.128 -0.166819
3 -18.652  93.481  0.263 -0.582301
4 -20.269  45.502  0.261 -0.568240


## 1d) Check for missing values and handle them if they exist

In [ ]:
# Checking for missing values
DF.isnull().sum() # looks like there are no missing values!

,0
TA,0
SW,0
VPD,0
GPP,0


# 2) Feature Engineering -- Interaction Terms (1 point)

## 2a) Create interaction terms between predictors
* SW x VPD
* SW x TA
* VPD x TA


In [7]:
DF['SW_VPD'] = DF['SW'] * DF['VPD'] # interaction term SW x VPD
DF['SW_TA'] = DF['SW'] * DF['TA'] # interaction term SW x TA
DF['VPD_TA'] = DF['VPD'] * DF['TA'] # interaction term VPD x TA

print(DF.head())  # printing dataframe head to confirm columns were added

       TA      SW    VPD       GPP     SW_VPD        SW_TA    VPD_TA
0 -20.063  72.603  0.413 -0.517364  29.985039 -1456.633989 -8.286019
1 -12.814  12.358  0.147 -0.094241   1.816626  -158.355412 -1.883658
2 -12.625  33.132  0.128 -0.166819   4.240896  -418.291500 -1.616000
3 -18.652  93.481  0.263 -0.582301  24.585503 -1743.607612 -4.905476
4 -20.269  45.502  0.261 -0.568240  11.876022  -922.280038 -5.290209


# 3) Build a Linear Regression Model using SciPy Optimization (3 points)

## Define the linear model equation then use MSE to minimize function to find the best fit parameters and print optimized parameters

GPP=β0​+β1​⋅SW+β2​⋅VPD+β3​⋅TA+β4​⋅(SW×VPD)+β5​⋅(SW×TA)+β6​⋅(VPD×TA)

In [14]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize

# Define the linear regression model function
def scipy_model(params, SW, VPD, TA):
    β0, β1, β2, β3, β4, β5, β6 = params
    return β0 + β1 * SW + β2 * VPD + β3 * TA + β4 * (SW * VPD) + β5 * (SW * TA) + β6 * (VPD * TA)

# Define the cost function (MSE)
def mse_loss(params, SW, VPD, TA, GPP):
    predictions = scipy_model(params, SW, VPD, TA)
    return np.mean((GPP - predictions) ** 2)

# Extract target variable
SW = DF['SW'].values
VPD = DF['VPD'].values
TA = DF['TA'].values
GPP = DF['GPP'].values

# Initial guess for the parameters
initial_params = np.ones(7)

# Minimize MSE using SciPy's minimize function
result = minimize(mse_loss, initial_params, args=(SW, VPD, TA, GPP))

# Extract optimized parameters
optimized_params = result.x

# Print the optimized coefficients
print("Optimized Coefficients:")
print(f"β0: {optimized_params[0]}")
print(f"β1: {optimized_params[1]}")
print(f"β2: {optimized_params[2]}")
print(f"β3: {optimized_params[3]}")
print(f"β4: {optimized_params[4]}")
print(f"β5: {optimized_params[5]}")
print(f"β6: {optimized_params[6]}")


DF['GPP_pred_scipy'] = scipy_model(optimized_params, DF['SW'], DF['VPD'], DF['TA'])
print(DF.head())

Optimized Coefficients:
β0: -944.5224632798087
β1: -3.2459289284319834
β2: -648.509157132203
β3: 82.16126152917857
β4: 1.7919500557851864
β5: -0.4560745445432938
β6: 13.598898674637196
       TA      SW    VPD       GPP     SW_VPD        SW_TA    VPD_TA  \
0 -20.063  72.603  0.413 -0.517364  29.985039 -1456.633989 -8.286019   
1 -12.814  12.358  0.147 -0.094241   1.816626  -158.355412 -1.883658   
2 -12.625  33.132  0.128 -0.166819   4.240896  -418.291500 -1.616000   
3 -18.652  93.481  0.263 -0.582301  24.585503 -1743.607612 -4.905476   
4 -20.269  45.502  0.261 -0.568240  11.876022  -922.280038 -5.290209   

   GPP_pred_sklearn  GPP_pred_scipy  
0      -2242.366835    -2491.037671  
1      -1854.574657    -2082.919403  
2      -1913.101230    -1995.965921  
3      -2264.494056    -2178.422934  
4      -2159.357561    -2556.837551  


# 4) Build a Linear Regression Model using Sklearn's Linear Regression Function (3 points)

Fit a linear regression model using Sklearn's LinearRegression Function then extract and print the regression coefficients for the model

In [15]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing, svm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# Defining predictor vs target variables

x = DF[['SW', 'VPD', 'TA']]
y = DF[['GPP']]

# Using the linear regression built in model
model = LinearRegression()
model.fit(x,y)

# New column for GPP_pred_sklearn
DF['GPP_pred_sklearn'] = model.predict(x)

# Print the coefficients and intercept
print("Intercept (β0):", model.intercept_)
print("Coefficients (β1, β2, β3):", model.coef_)

# Checking to see if predictions are added to dataframe
print(DF.head())

Intercept (β0): [-1470.0381342]
Coefficients (β1, β2, β3): [[ -3.08877295 -31.54033557  26.6684125 ]]
       TA      SW    VPD       GPP     SW_VPD        SW_TA    VPD_TA  \
0 -20.063  72.603  0.413 -0.517364  29.985039 -1456.633989 -8.286019   
1 -12.814  12.358  0.147 -0.094241   1.816626  -158.355412 -1.883658   
2 -12.625  33.132  0.128 -0.166819   4.240896  -418.291500 -1.616000   
3 -18.652  93.481  0.263 -0.582301  24.585503 -1743.607612 -4.905476   
4 -20.269  45.502  0.261 -0.568240  11.876022  -922.280038 -5.290209   

   GPP_pred_sklearn  GPP_pred_scipy  
0      -2242.366835    -2491.037671  
1      -1854.574657    -2082.919403  
2      -1913.101230    -1995.965921  
3      -2264.494056    -2178.422934  
4      -2159.357561    -2556.837551  


# 5) Compare the Models (2 points)

## Compute the R-squared for both models and compare the coefficients obtained from scipy's optimization model and sklearn's LinearRegression function

R-squared formula:

1 - [(actual - predicted)^2 / (actual - mean)^2]

In [19]:
# For scipy_model:

# Compute R-squared for scipy model:
y_actual = DF['GPP']
y_pred_scipy = DF['GPP_pred_scipy']
Rsquared_scipy = (np.sum((y_actual - y_pred_scipy) ** 2) / np.sum((y_actual - np.mean(y_actual)) ** 2))
print(Rsquared_scipy)

# Compute R-squared for sklearn model:
y_pred_sklearn = DF['GPP_pred_sklearn']
Rsquared_sklearn = (np.sum((y_actual - y_pred_sklearn) ** 2) / np.sum((y_actual - np.mean(y_actual)) ** 2))
print(Rsquared_sklearn)

0.9864173011216913
0.9945893788825403
